# RoI and LoI analysis (2D experiments only)

In [1]:
import pickle5
import statistics

In [2]:
# get required information from user
DataPath = "../../2_HN168107_PBI_PBII_v07_600K_Min2Ratio2/" # the folder containing the required files
OptFN = f"{DataPath}options_full.ini"

In [3]:
#### FUNCTIONS ####
# get run parameters from options file	
def get_Options(OptFN, OptionsDic, AllowedPars):
    try:
        optionsfile=open(OptFN, "r")
        for line in optionsfile :
            if line[:1] != "#" and len(line.strip("\n".strip(""))) > 1 :
                splitted_line = line.split(":")
                key=splitted_line[0].split(" ")[0]
                value=splitted_line[1].strip("\n").split(" ")[0]
                if key in AllowedPars :
                    OptionsDic[key] = value 
    except:
        print('Error reading %s file !' % (OptFN))

    return OptionsDic

#retrieve list of lists based on 2 separators
def double_split(Text, sep1, sep2):
    # further parse options
    WholeL = []
    MainL = Text.split(sep1)
    for text in MainL:
        WholeL.append(text.strip().split(sep2))

    return WholeL

In [4]:
## load required data
# the options
OptionsDic = {}
AllowedPars = ["Partners"]
OptionsDic = get_Options(OptFN, OptionsDic, AllowedPars)
ComplexesL = double_split(OptionsDic["Partners"], ";", "~")[0]
# the AAvarsDic
AAvarsDic = {} 
with open(r"{}SR_AAvars_{}_vs_{}.pkl".format(DataPath,ComplexesL[0], ComplexesL[1]), "rb") as input_file:
    AAvarsDic = pickle5.load(input_file)
    
## Drop irrelevant information
for key in AAvarsDic.keys():
    del AAvarsDic[key]['IndBCData']
    
print("{} single mutant combinations found in SR_AAvars dictionary for complex partners {} (A) and {} (B) !".format(len(AAvarsDic.keys()), ComplexesL[0], ComplexesL[1]))

12275 single mutant combinations found in SR_AAvars dictionary for complex partners VN1551_VHH2 (A) and VN1554_TNFa-2 (B) !


In [5]:
# RoI analysis I
RoIDic = {}
AoBxWT_MaxNormEnrich = 0.7 # maximum allowed enrichment of a single mutant partner against le WT
WT_MinDeltaMax = 0.2 # the minimum difference between a combination of single mutant partners and their combinations (each) with WT

for key in AAvarsDic.keys():
    if int(AAvarsDic[key]['Stats'][2]) >= 6  :
        if AAvarsDic[key]['Stats'][1] <= AAvarsDic[key]['Stats'][0] * 0.7 :
            split_key = key.split(":")
            Partner_A = split_key[0]
            Partner_B = split_key[1]
            if (Partner_A != 'WT') and (Partner_B != 'WT')  and ( f"{Partner_A}:WT" in AAvarsDic.keys() ) and ( f"WT:{Partner_B}" in AAvarsDic.keys() ):
                AxWTb = AAvarsDic[f"{Partner_A}:WT"]['Stats'][0]
                BxWTa = AAvarsDic[f"WT:{Partner_B}"]['Stats'][0]
                if (AxWTb <= AoBxWT_MaxNormEnrich) or (BxWTa <= AoBxWT_MaxNormEnrich):
                    AxB = AAvarsDic[key]['Stats'][0]
                    DeltaA = AxB - AxWTb 
                    DeltaB = AxB - BxWTa
                    DeltaABL = [ DeltaA, DeltaB]
                    DeltaMax = max(DeltaABL)
                    DeltaMean = statistics.mean(DeltaABL)
                    AxWTname = f"{Partner_A}:WT"
                    BxWTname = f"WT:{Partner_B}"
                    AxWT_BCs =  AAvarsDic[AxWTname]['Stats'][2]
                    BxWT_BCs =  AAvarsDic[BxWTname]['Stats'][2]
                    AxB_BCs = AAvarsDic[key]['Stats'][2]
                    if DeltaMax >= WT_MinDeltaMax:
                        RoIDic[key] = [AxWTb, BxWTa, AxB, DeltaMean, DeltaMax, AxWT_BCs, BxWT_BCs, AxB_BCs ]

sorted_RoIDic = sorted(RoIDic.items(), key=lambda item: item[1][-1], reverse=True)
for n in range(len(sorted_RoIDic)):
    print(f"{sorted_RoIDic[n][0]} => recovery= {sorted_RoIDic[n][1][4]:.3}, AxWt= {sorted_RoIDic[n][1][0]:.3} ({sorted_RoIDic[n][1][5]}), BxWt= {sorted_RoIDic[n][1][1]:.3} ({sorted_RoIDic[n][1][6]}), AxB= {sorted_RoIDic[n][1][2]:.3} ({sorted_RoIDic[n][1][7]})")

F103A:E146S => recovery= 0.3, AxWt= 1.07 (252), BxWt= 0.627 (159), AxB= 0.928 (17)
W33R:N92D => recovery= 0.344, AxWt= 0.118 (174), BxWt= 0.896 (175), AxB= 0.462 (15)
Y32E:E146S => recovery= 0.346, AxWt= 1.12 (174), BxWt= 0.627 (159), AxB= 0.974 (11)
S101E:E146S => recovery= 0.3, AxWt= 1.14 (168), BxWt= 0.627 (159), AxB= 0.927 (10)
S101T:E146S => recovery= 0.374, AxWt= 1.14 (113), BxWt= 0.627 (159), AxB= 1.0 (8)
S101T:T89L => recovery= 0.313, AxWt= 1.14 (113), BxWt= 0.623 (129), AxB= 0.936 (7)
S101A:E146S => recovery= 0.233, AxWt= 1.06 (118), BxWt= 0.627 (159), AxB= 0.861 (7)


In [6]:
# RoI analysis II
RoIDic = {}
AoBxWT_MinNormEnrich = 0.9 # minimum allowed enrichment of a single mutant partner against le WT
WT_MinDeltaMax = 0.3 # the minimum difference between a combination of single mutant partners and their combinations (each) with WT

for key in AAvarsDic.keys():
    if int(AAvarsDic[key]['Stats'][2]) >= 6  :
        if AAvarsDic[key]['Stats'][1] <= AAvarsDic[key]['Stats'][0] *0.7 :
            split_key = key.split(":")
            Partner_A = split_key[0]
            Partner_B = split_key[1]
            if (Partner_A != 'WT') and (Partner_B != 'WT')  and ( "{}:WT".format(Partner_A) in AAvarsDic.keys() ) and ( "WT:{}".format(Partner_B) in AAvarsDic.keys() ):
                AxWTb = AAvarsDic["{}:WT".format(Partner_A)]['Stats'][0]
                BxWTa = AAvarsDic["WT:{}".format(Partner_B)]['Stats'][0]
                if (AxWTb >= AoBxWT_MinNormEnrich) and (BxWTa >= AoBxWT_MinNormEnrich):
                    AxB = AAvarsDic[key]['Stats'][0]
                    DeltaA = AxB - AxWTb 
                    DeltaB = AxB - BxWTa
                    DeltaABL = [ DeltaA, DeltaB]
                    DeltaMax = max(DeltaABL)
                    DeltaMean = statistics.mean(DeltaABL)
                    AxWTname = f"{Partner_A}:WT"
                    BxWTname = f"WT:{Partner_B}"
                    AxWT_BCs =  AAvarsDic[AxWTname]['Stats'][2]
                    BxWT_BCs =  AAvarsDic[BxWTname]['Stats'][2]
                    AxB_BCs = AAvarsDic[key]['Stats'][2]
                    if DeltaMax >= WT_MinDeltaMax:
                        RoIDic[key] = [AxWTb, BxWTa, AxB, DeltaMean, DeltaMax, AxWT_BCs, BxWT_BCs, AxB_BCs ]


sorted_RoIDic = sorted(RoIDic.items(), key=lambda item: item[1][-1], reverse=True)
for n in range(len(sorted_RoIDic)):
    print(f"{sorted_RoIDic[n][0]} => recovery= {sorted_RoIDic[n][1][4]:.3}, AxWt= {sorted_RoIDic[n][1][0]:.3} ({sorted_RoIDic[n][1][5]}), BxWt= {sorted_RoIDic[n][1][1]:.3} ({sorted_RoIDic[n][1][6]}), AxB= {sorted_RoIDic[n][1][2]:.3} ({sorted_RoIDic[n][1][7]})")

Y32E:S147F => recovery= 0.33, AxWt= 1.12 (174), BxWt= 0.962 (116), AxB= 1.29 (10)
S30G:Q88N => recovery= 0.379, AxWt= 1.1 (96), BxWt= 1.08 (168), AxB= 1.46 (8)
S101F:S147F => recovery= 0.302, AxWt= 1.14 (121), BxWt= 0.962 (116), AxB= 1.26 (8)
Y35W:S147E => recovery= 0.362, AxWt= 0.92 (215), BxWt= 1.16 (58), AxB= 1.28 (7)
Y32E:N92A => recovery= 0.387, AxWt= 1.12 (174), BxWt= 0.92 (123), AxB= 1.31 (6)
F103R:N92A => recovery= 0.342, AxWt= 1.11 (136), BxWt= 0.92 (123), AxB= 1.26 (6)


In [7]:
# LoI analysis I
LoIDic = {}
AoBxWT_MinNormEnrich = 0.9 # minimum allowed enrichment of a single mutant partner against le WT
WT_MinDeltaMin = -0.25 # the minimum difference between a combination of single mutant partners and their combinations (each) with WT

for key in AAvarsDic.keys():
    if int(AAvarsDic[key]['Stats'][2]) >= 6  :
        if AAvarsDic[key]['Stats'][1] <= AAvarsDic[key]['Stats'][0] *0.7 :
            split_key = key.split(":")
            Partner_A = split_key[0]
            Partner_B = split_key[1]
            if (Partner_A != 'WT') and (Partner_B != 'WT')  and ( "{}:WT".format(Partner_A) in AAvarsDic.keys() ) and ( "WT:{}".format(Partner_B) in AAvarsDic.keys() ):
                AxWTb = AAvarsDic["{}:WT".format(Partner_A)]['Stats'][0]
                BxWTa = AAvarsDic["WT:{}".format(Partner_B)]['Stats'][0]
                if (AxWTb >= AoBxWT_MinNormEnrich) and (BxWTa >= AoBxWT_MinNormEnrich):
                    AxB = AAvarsDic[key]['Stats'][0]
                    DeltaA = AxB - AxWTb 
                    DeltaB = AxB - BxWTa
                    DeltaABL = [ DeltaA, DeltaB]
                    DeltaMin = min(DeltaABL)
                    DeltaMean = statistics.mean(DeltaABL)
                    AxWTname = f"{Partner_A}:WT"
                    BxWTname = f"WT:{Partner_B}"
                    AxWT_BCs =  AAvarsDic[AxWTname]['Stats'][2]
                    BxWT_BCs =  AAvarsDic[BxWTname]['Stats'][2]
                    AxB_BCs = AAvarsDic[key]['Stats'][2]                    
                    if DeltaMin <= WT_MinDeltaMin:
                        LoIDic[key] = [AxWTb, BxWTa, AxB, DeltaMean, DeltaMin, AxWT_BCs, BxWT_BCs, AxB_BCs ]
                        

sorted_LoIDic = sorted(LoIDic.items(), key=lambda item: item[1][-1], reverse=False)
for n in range(len(sorted_LoIDic)):
    print(f"{sorted_LoIDic[n][0]} => loss= {sorted_LoIDic[n][1][4]:.3}, AxWt= {sorted_LoIDic[n][1][0]:.3} ({sorted_LoIDic[n][1][5]}), BxWt= {sorted_LoIDic[n][1][1]:.3} ({sorted_LoIDic[n][1][6]}), AxB= {sorted_LoIDic[n][1][2]:.3} ({sorted_LoIDic[n][1][7]}) ")

R98E:Q88N => loss= -0.361, AxWt= 0.979 (79), BxWt= 1.08 (168), AxB= 0.724 (6) 
R98L:Q88N => loss= -0.267, AxWt= 1.33 (75), BxWt= 1.08 (168), AxB= 1.06 (6) 
L56Q:S81T => loss= -0.412, AxWt= 1.02 (117), BxWt= 0.934 (105), AxB= 0.605 (6) 
R98E:T77E => loss= -0.503, AxWt= 0.979 (79), BxWt= 0.995 (146), AxB= 0.491 (7) 
R98G:S81T => loss= -0.415, AxWt= 0.953 (188), BxWt= 0.934 (105), AxB= 0.539 (8) 
R98G:Q88K => loss= -0.344, AxWt= 0.953 (188), BxWt= 0.979 (70), AxB= 0.636 (8) 
F103A:T79S => loss= -0.288, AxWt= 1.07 (252), BxWt= 0.913 (60), AxB= 0.778 (8) 
S30R:Q88N => loss= -0.262, AxWt= 1.09 (81), BxWt= 1.08 (168), AxB= 0.832 (9) 
I57V:T77E => loss= -0.296, AxWt= 1.07 (196), BxWt= 0.995 (146), AxB= 0.775 (13) 
R98G:T77E => loss= -0.381, AxWt= 0.953 (188), BxWt= 0.995 (146), AxB= 0.614 (14) 
